In [2]:
# coding=UTF-8

#importando os modulos necessarios e criando as funções para preparar os dados

import re
from unicodedata import normalize
from nltk import FreqDist
from nltk import tokenize
import nltk
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.probability import FreqDist
from nltk.stem.snowball import SnowballStemmer
from sklearn import metrics
from sklearn import svm
from sklearn import tree
import sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

import pandas as pd

def lerCSV(nomeArquivo):
    return pd.read_csv(nomeArquivo, encoding='ISO-8859-1', sep=";", header=0)

def removerPontuacao(texto):
    return re.sub(u'[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]', ' ', texto)

def removerStopWord(texto):
    stopWord=nltk.corpus.stopwords.words('portuguese')
    sentencaSemStopword = [i for i in texto.split() if not i in stopWord]
    return " ".join(sentencaSemStopword)

def removerAssentuacao(texto):
    return normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')

def aplicarTokenize(texto):
    return tokenize.word_tokenize(texto, language='portuguese')

def removerNumeros(texto):
    return re.sub('[0-9]', '', texto)

def removerURL(texto):
    return re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*','', texto)

def transformarEmMinusculas(texto):
    return texto.lower()

def aplicarStemming(texto):
    texto=str(texto)
    stemmer = nltk.stem.RSLPStemmer()
    palavras=[]
    for txt in texto.split():
        palavra=stemmer.stem(txt)
        palavras.append(palavra)
    return " ".join(palavras)

def removerColuna(dataframe):
    dataframe=dataframe.drop(dataframe.columns[0], axis=1)
    return dataframe

def removerMensionamento(texto):
    if texto.find("@")>=0:
        texto=texto.split("@")
        texto=str(texto[-1])
        contem=texto.find(" ")
        texto=texto[contem+1:]
        
    return texto

def etl(textos):
    
    tweets=[]
   
    #trata o texto de cada linha do dataframe
    for texto in textos:
        texto=str(texto)
        
        #remover mensionamento (@usuario)
        texto=removerMensionamento(texto)
        
        #remove URL
        texto=removerURL(texto)
        
        #transforma a frase toda em minúscula
        texto=transformarEmMinusculas(texto)
        
        #remove números
        texto=removerNumeros(texto)
        
        #remove as pontuações
        texto=removerPontuacao(texto)
        
        #retirar assentos
        texto=removerAssentuacao(texto)
        
        #aplica stemming
        texto=aplicarStemming(texto)
        
        #remove stopwords
        texto=removerStopWord(texto)
        
        texto=str(texto)
        
        tweets.append(texto)
    return tweets

def classificarDecisionTree(texto, sentimento):
    print("\nAlgoritmo decision tree")
    
    #cria um vetor de 1 palavra
    vetor=criarVetor1Palavra()
    
    #pega a frequência das palavras
    texto_freq=vetor.fit_transform(texto)
    
    #cria o modelo
    modelo = tree.DecisionTreeClassifier()
    
    # Criando uma Confusion Matrix
    avaliarModelo(modelo, texto_freq, sentimento)

def classificarSVM(texto, sentimento):
    print("\nAlgoritmo svm:")
    
    #cria um vetor de 1 palavra
    vetor = criarVetor1Palavra()
    
    #pega a frequência das palavras
    texto_freq = vetor.fit_transform(texto)
    
    #cria o modelo
    modelo = svm.SVC(gamma=0.001, C=100.)
    
    # Criando uma Confusion Matrix
    avaliarModelo(modelo, texto_freq, sentimento)

def classificarMultinomialNB(texto, sentimento):
    print("\nAlgoritmo NaiveBayes:")
    
        #cria um vetor de 1 palavra
    vetor = CountVectorizer(analyzer="word")
    
    #pega a frequência das palavras
    texto_freq = vetor.fit_transform(texto)
    
    #cria o modelo
    modelo = MultinomialNB()
    
    # Criando uma Confusion Matrix
    avaliarModelo(modelo, texto_freq, sentimento)

def criarVetor1Palavra():
        #a linha abaixo traz o vetor de 1 palavra
    return CountVectorizer(analyzer="word")

def avaliarModelo(modelo, texto, sentimento):
    resultados = cross_val_predict(modelo, texto, sentimento, cv=10)
    
    #calcula acurácia
    acuracia=metrics.accuracy_score(sentimento, resultados)    
    print("acurácia (cross validation): {:.2f}".format(acuracia))
    print("matriz de confusão")
    print("INSEGURO - OUTRO")
    print("{}".format(metrics.confusion_matrix(sentimento, resultados.ravel())))

def classeNumerica(df):
    sentimento_map = {"inseguro": 0, "outro": 1}
    return df.map(sentimento_map)

def classificarLogisticRegression(texto, sentimento):
    print("\nAlgoritmo regressão logística:")
    
        #cria um vetor de 1 palavra
    vetor = criarVetor1Palavra()
    
    #pega a frequência das palavras
    texto_freq = vetor.fit_transform(texto)
    
    #cria o modelo
    modelo=LogisticRegression()
    
    # Criando uma Confusion Matrix
    avaliarModelo(modelo, texto_freq, sentimento)

def classificarRandomForestClassifier(texto, sentimento):
    print("\nAlgoritmo floresta aleatória:")
    
    #cria um vetor de 1 palavra
    vetor=criarVetor1Palavra()
    
    #pega a frequência das palavras
    texto_freq=vetor.fit_transform(texto)
    
    #cria o modelo
    modelo = RandomForestClassifier(random_state = 42)
    
    # Criando uma Confusion Matrix
    avaliarModelo(modelo, texto_freq, sentimento)


In [3]:
def main():
    #base de tweets classificados
    nomeArquivo="tweets_classificados.csv"
    
    #ler a base de tweets classificados
    df=lerCSV(nomeArquivo)
    
    # pega 200 inseguros e 200 outros
    inseguros=df[df['sentimento']=="inseguro"]
    inseguros=inseguros.head(200)
    outros=df[df['sentimento']=="outro"]
    outros=outros.head(200)
    
    #concatena os dois dataframes
    dfn=inseguros.append(outros, ignore_index=True)
    
    #remove a coluna usuário do dataframe
    dft=removerColuna(dfn)
    
    #transforma a coluna sentimento de string para número
    dft['sentimento']=classeNumerica(dft['sentimento'])
    
    #trata os textos
    dft['texto']=etl(dft['texto'])
    
    classificarSVM(dft['texto'], dft['sentimento'])

main()


Algoritmo svm:
acurácia (cross validation): 0.67
matriz de confusão
INSEGURO - OUTRO
[[149  51]
 [ 82 118]]


In [4]:
def main():
    #base de tweets classificados
    nomeArquivo="tweets_classificados.csv"
    
    #ler a base de tweets classificados
    df=lerCSV(nomeArquivo)
    
    # pega 200 inseguros e 200 outros
    inseguros=df[df['sentimento']=="inseguro"]
    inseguros=inseguros.head(200)
    outros=df[df['sentimento']=="outro"]
    outros=outros.head(200)
    
    #concatena os dois dataframes
    dfn=inseguros.append(outros, ignore_index=True)
    
    #remove a coluna usuário do dataframe
    dft=removerColuna(dfn)
    
    #transforma a coluna sentimento de string para número
    dft['sentimento']=classeNumerica(dft['sentimento'])
    
    #trata os textos
    dft['texto']=etl(dft['texto'])
    
    classificarDecisionTree(dft['texto'], dft['sentimento'])

main()


Algoritmo decision tree
acurácia (cross validation): 0.59
matriz de confusão
INSEGURO - OUTRO
[[137  63]
 [ 99 101]]


In [5]:
def main():
    #base de tweets classificados
    nomeArquivo="tweets_classificados.csv"
    
    #ler a base de tweets classificados
    df=lerCSV(nomeArquivo)
    
    # pega 200 inseguros e 200 outros
    inseguros=df[df['sentimento']=="inseguro"]
    inseguros=inseguros.head(200)
    outros=df[df['sentimento']=="outro"]
    outros=outros.head(200)
    
    #concatena os dois dataframes
    dfn=inseguros.append(outros, ignore_index=True)
    
    #remove a coluna usuário do dataframe
    dft=removerColuna(dfn)
    
    #transforma a coluna sentimento de string para número
    dft['sentimento']=classeNumerica(dft['sentimento'])
    
    #trata os textos
    dft['texto']=etl(dft['texto'])
    
    classificarMultinomialNB(dft['texto'], dft['sentimento'])

main()


Algoritmo NaiveBayes:
acurácia (cross validation): 0.61
matriz de confusão
INSEGURO - OUTRO
[[151  49]
 [105  95]]


In [6]:
def main():
    #base de tweets classificados
    nomeArquivo="tweets_classificados.csv"
    
    #ler a base de tweets classificados
    df=lerCSV(nomeArquivo)
    
    # pega 200 inseguros e 200 outros
    inseguros=df[df['sentimento']=="inseguro"]
    inseguros=inseguros.head(200)
    outros=df[df['sentimento']=="outro"]
    outros=outros.head(200)
    
    #concatena os dois dataframes
    dfn=inseguros.append(outros, ignore_index=True)
    
    #remove a coluna usuário do dataframe
    dft=removerColuna(dfn)
    
    #transforma a coluna sentimento de string para número
    dft['sentimento']=classeNumerica(dft['sentimento'])
    
    #trata os textos
    dft['texto']=etl(dft['texto'])
    
    classificarLogisticRegression(dft['texto'], dft['sentimento'])

main()


Algoritmo regressão logística:
acurácia (cross validation): 0.67
matriz de confusão
INSEGURO - OUTRO
[[151  49]
 [ 84 116]]


In [7]:
def main():
    #base de tweets classificados
    nomeArquivo="tweets_classificados.csv"
    
    #ler a base de tweets classificados
    df=lerCSV(nomeArquivo)
    
    # pega 200 inseguros e 200 outros
    inseguros=df[df['sentimento']=="inseguro"]
    inseguros=inseguros.head(200)
    outros=df[df['sentimento']=="outro"]
    outros=outros.head(200)
    
    #concatena os dois dataframes
    dfn=inseguros.append(outros, ignore_index=True)
    
    #remove a coluna usuário do dataframe
    dft=removerColuna(dfn)
    
    #transforma a coluna sentimento de string para número
    dft['sentimento']=classeNumerica(dft['sentimento'])
    
    #trata os textos
    dft['texto']=etl(dft['texto'])
    
    classificarRandomForestClassifier(dft['texto'], dft['sentimento'])

main()


Algoritmo floresta aleatória:
acurácia (cross validation): 0.65
matriz de confusão
INSEGURO - OUTRO
[[138  62]
 [ 77 123]]
